# Objects Expression recognition

This notebooks shows some preliminary work on detecting objects expressions with cvlib. It is mainly meant to explore its capabilities and to decide on future research directions. We package our code into a `misinformation` package that is imported here:

In [ ]:
import misinformation
from misinformation import utils as mutils
from misinformation import display as mdisplay
import misinformation.objects as ob

Set an image path as input file path.

In [ ]:
images = mutils.find_files(
    path="data/",
    limit=10,
)

In [ ]:
mydict = mutils.initialize_dict(images)

## Detect objects and directly write to csv

In [ ]:
for key in mydict:
    mydict[key] = ob.ObjectDetector(mydict[key]).analyse_image()

Convert the dictionary of dictionarys into a dictionary with lists:

In [ ]:
outdict = mutils.append_data_to_dict(mydict)
df = mutils.dump_df(outdict)

Check the dataframe:

In [ ]:
df.head(10)

Write the csv file:

In [ ]:
df.to_csv("./data_out.csv")

## Manually inspect what was detected

To check the analysis, you can inspect the analyzed elements here. Loading the results takes a moment, so please be patient. If you are sure of what you are doing.

In [ ]:
mdisplay.explore_analysis(mydict, identify="objects")

In [ ]:
def localize_objects(path):
    """Localize objects in the local image.

    Args:
    path: The path to the local file.
    """
    from google.cloud import vision

    client = vision.ImageAnnotatorClient()

    with open(path, "rb") as image_file:
        content = image_file.read()
    image = vision.Image(content=content)

    objects = client.object_localization(image=image).localized_object_annotations

    print("Number of objects found: {}".format(len(objects)))
    for object_ in objects:
        print("\n{} (confidence: {})".format(object_.name, object_.score))
        print("Normalized bounding polygon vertices: ")
        for vertex in object_.bounding_poly.normalized_vertices:
            print(" - ({}, {})".format(vertex.x, vertex.y))

In [ ]:
import os

os.environ[
    "GOOGLE_APPLICATION_CREDENTIALS"
] = "../../misinformation-notes/seismic-bonfire-329406-412821a70264.json"
localize_objects("/home/iulusoy/Desktop/102141_2_eng.png")